# Lecture 5: simulating quantum noise

<center><img src="../figures/qibo_mascotte/qibo_noise.png" alt="drawing" width="400"/></center>
<center><strong>Fig. 5:</strong> Qibo the mangoose suffering noise [DALL-E].</center>

## Introduction

In real life scenarios, quantum computers are imperfect and prone to noise. Some of the noise sources can be:
- **decoerence**, which corresponds to the loss of the quantum properties of the system, usually due to some interaction with the external environment;
- **thermal noise**, which affects in particular systems sensitive to temperature;
- **control errors**, due to some imperfections in the laboratory tools (cables, control boards, etc.);
- **cross-talk**, which happens when some operation we apply to a single qubit unintentionally affect the neighbouring qubits;
- **quantum state leakage**, which is the possibility to access some undesired energy level. It can happen for example using superconducting qubits, which are in principle N-levels systems prepared to isolate the first two energy levels.

More in general, a combination of these phenomena can affect a real quantum device, leading to a simple but relevant problem: the expected results are corrupted. 

In `qibo`, we can simulate noisy systems and implement some strategies to take this problem into account, cleaning our results from undesired corruptions. 

In the following:

1. we define a simple problem, consisting in calculating the expected value of a target observable over the final state we obtain executing a quantum circuit;
2. we compute the exact result with noiseless simulation;
3. we add noise to the circuit and compute the noisy result;

#### 1. Problem definition

We consider as an example a simple quantum system of three qubits, on which we aim to apply some rotations and then compute the expectation value of an observable composed of Pauli's.

In [ ]:
# if you don't have already qibo, qibojit and qiboedu installed in your computer
# uncomment and execute the following lines 

# !pip install qibo
# !pip install qibojit
# !pip install git+https://github.com/qiboteam/qiboedu

In [ ]:
# import numpy and matplotlib

# import qibo, Circuit, gates, hamiltonians

# import plotscripts from qiboedu

In [ ]:
# set qibo's backend

In [ ]:
# define a parametric circuit with layered architecture
# set density_matrix=True when initializing the circuit

In [ ]:
# draw the circuit

Now the circuit is set up and we can fix its action by injecting a specific list of rotational angles. 

We get the numbers of contained parameters:

In [ ]:
# get the number of parameters of the circuit

And then we generate a well defined set of angles.

In [ ]:
# fix the set of angles

In [ ]:
# set the angles into the circuit

Now we have a quantum circuit composed of rotations and some CNOT gates. We want to use it to compute some calculations. 

To do this, we can define an observable $\mathcal{O}$, for which we calculate:

$$ \langle \mathcal{O} \rangle \equiv \langle 0 | U^{\dagger}\, \mathcal{O}\, U | 0 \rangle \\. $$

We make the choice:

$$ \mathcal{O} = - \sum_{i=0}^N Z_i \\.$$

In [ ]:
# observable definition

#### 2. Computing the exact expectation value

In [ ]:
# execute the circuit

# print final state

# collect and visualize frequencies

Let's have a look also to the density matrix of this system.
We can use the function `plotscript.plot_density_matrix(state)`, which takes as argument the state object you get from a circuit execution `circuit().state()`. It is important to set the variable `density_matrix=True` when initializing the circuit.

The plot is going to show the absolute value of the amplitudes according to the density matrix 2D notation.

In [ ]:
# plot the density matrix using plotscripts.plot_density_matrix(state)

As we can see from the plot, we have both diagonal and non diagonal components in the matrix.

The expectation value of this observable, which is diagonal in the computational basis can be computed directly from the frequencies. 

Executing the system, we collect one frequency for each component of the state vector. We call $f_i$ the frequency corresponding to the $i-$th component of the state vector. 

Now, considering an observable $\mathcal{O}$ whose eigenvalues are $\{o_i\}_{i=1}^{2^N}$, the expectation value of $\mathcal{O}$ given the set of frequencies $\{f_i\}_{i=1}^{2^N}$ is:

$$ E[\mathcal{O}] = \frac{1}{2^N} \sum_{i=1}^{2^N} o_i\, f_i. $$

In `Qibo`, this formula is implemented into a method of the `Hamiltonian` object:
`observable.expectation_from_samples(frequencies)`.

In [ ]:
# compute the expected value with expectation_from_samples

#### 3. Add noise to the system

Various different ways to simulate noise exist in `qibo`, but in this lecture we focus on one of them. 

Let me introduce before the Pauli noise channel, which is used here to corrupt the circuit. If we consider a quantum state represented by the density matrix $\rho$, the effect of a Pauli noise channel on the state is

$$ \mathcal{E}(\rho) = \biggl(1 - \sum_{k=0}p_k\biggr)\rho + \sum_k p_k P_k \,\rho\, P_k \\,
$$

where $P_k$ is the $k$-th Pauli string and $p_k$ is the probability of applying $P_k$. 

This channel allows to formalize a corruption of the state $\rho$ in which each Pauli represents some specific error which can occurr: for example, the Pauli X and Z respectively represent a bit-flip error and a phase-flip error in the qubit state.

We are going to use the `circuit.with_pauli_noise` method, which consists in applying a Pauli noise channel after every gate of the circuit. 

In [ ]:
# define the Pauli's probabilities for each qubit of the system

# use the circuit.with_pauli_noise

Let's print the new face of the circuit, with the Pauli noise channel after each gate.

In [ ]:
# draw the circuit

In [ ]:
# set the parameters

In [ ]:
# execute the circuit

# collect and visualize frequencies

In [ ]:
# plot the density matrix using plotscripts.plot_density_matrix(state)

From the matrix, we can see the off diagonal terms are vanishing.

As final step of this lecture 4, let's compute the expectation value of $\mathcal{O}$ over the noisy final state.

In [ ]:
# compare the expectation values with and without noise

<div style="background-color: rgba(255, 105, 105, 0.3); border: 2.5px solid #000000; padding: 15px;">
    <strong>Exercise:</strong> what is happening to the system? why is the expected value of $Z$ decreasing? What happens if you increase the magnitude of the noise?
</div>